<a href="https://colab.research.google.com/github/iamrajatroy/Data-Science-Lab/blob/main/notebook/PandasAI_Agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# !pip install pandasai chromadb qdrant-client protobuf==3.20.3

In [37]:
import os
import pandas as pd
from pandasai.agent import Agent
from pandasai.llm.openai import OpenAI
from pandasai.ee.vectorstores import ChromaDB

In [3]:
os.environ["OPENAI_API_KEY"] = "*****"
os.environ["PANDASAI_API_KEY"] = "*****"

In [4]:
train_file_path = "/content/sample_data/california_housing_train.csv"

In [5]:
df = pd.read_csv(train_file_path)

In [6]:
df= df.sample(n=100, random_state=42)

In [7]:
df.shape

(100, 9)

In [8]:
df

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value
10941,-120.87,37.77,9.0,4838.0,920.0,2460.0,923.0,3.5959,142700.0
5250,-118.14,34.11,52.0,2742.0,422.0,1153.0,414.0,8.1124,500001.0
10292,-120.05,36.98,16.0,3705.0,739.0,2463.0,697.0,2.5288,61800.0
2266,-117.42,34.02,9.0,5455.0,882.0,3015.0,858.0,4.2321,162800.0
6398,-118.26,33.97,52.0,1331.0,346.0,1144.0,362.0,1.5326,90600.0
...,...,...,...,...,...,...,...,...,...
3260,-117.86,33.75,39.0,275.0,87.0,554.0,103.0,3.5972,158000.0
4930,-118.10,33.82,36.0,1930.0,354.0,915.0,328.0,5.2713,244400.0
5007,-118.11,33.83,36.0,1784.0,303.0,964.0,299.0,4.2703,220900.0
15132,-122.26,37.84,52.0,696.0,191.0,345.0,174.0,2.6736,191300.0


In [83]:
llm = OpenAI(model="gpt-4", temperature=0.0)

In [96]:
agent = Agent(df, vectorstore=ChromaDB(), config={"llm": llm})

In [97]:
query = "What should be value of a house with less than 9 years of age?"

code = """
## Given the query asked by user you need to generate the code in following format.
import pandas as pd
df = dfs[0]


## Filter the dataframe by relevant column
## Add additional filters by column based on the query
df_lt_9_yrs = df[df['housing_median_age'] < 9]

## get the average value of the house
## Always provide the result in the following format do not add return statement
result = {"type": "number", "value": df_lt_9_yrs['median_house_value'].mean()}

## Strictly do not add any return statement in the end

"""

In [98]:
agent.train(queries=[query], codes=[code])

In [16]:
agent.chat("What should be the value of the house with greater than 10 years of age?")

218384.27368421052

In [17]:
print(agent.context.intermediate_values['current_code_executed'])

# TODO: import the required dependencies
import pandas as pd

# Write code here
df = dfs[0]

df_gt_10_yrs = df[df['housing_median_age'] > 10]

result = {"type": "number", "value": df_gt_10_yrs['median_house_value'].mean()}


In [102]:
agent.chat("What should be the value of the house with greater than 10 years of age and with rooms less than 500 capacity?")

177825.0

In [103]:
print(agent.context.intermediate_values['current_code_executed'])

# TODO: import the required dependencies
import pandas as pd

# Write code here
df = dfs[0]

def get_price(df):
    ## Filter the dataframe by relevant columns 
    df_gt_10_yrs = df[(df['housing_median_age'] > 10) & (df['total_rooms'] < 500)]

    ## get the average value of the house
    results = {"house_value": df_gt_10_yrs['median_house_value'].mean()}

    ## return a dictionary of results
    return results

result = get_price(df)
result = { "type": "number", "value": result['house_value'] }
